In [ ]:
import cv2
import os
import requests
import socket
import threading

# Roboflow API Configuration
API_URL = "https://classify.roboflow.com/identifying-egyptian-artifacts/2"
API_KEY = "b5eHFI0497xgmPVPZxzh"

def capture_image(cap, image_path="captured_image.jpg"):
    """Capture an image from the webcam."""
    ret, frame = cap.read()
    if ret:
        cv2.imwrite(image_path, frame)
        return frame
    else:
        raise Exception("Failed to capture image from webcam")

def infer_image(image_path):
    """Send an image to the Roboflow API for inference."""
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image file '{image_path}' not found.")

    with open(image_path, "rb") as img_file:
        response = requests.post(
            f"{API_URL}?api_key={API_KEY}",
            files={"file": img_file}
        )
    
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"API request failed: {response.status_code}, {response.text}")

def update_predictions(frame, predictions):
    """Overlay predictions on the frame."""
    if predictions:
        label = predictions[0]['class']
        confidence = predictions[0]['confidence']
        if confidence > 0.1:
            cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else:
            cv2.putText(frame, "Unknown", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    else:
        cv2.putText(frame, "Unknown", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    return frame

def perform_inference_and_update(cap):
    """Capture an image, perform inference, and return updated predictions."""
    frame = capture_image(cap)
    result = infer_image("captured_image.jpg")
    predictions = result.get('predictions', [])
    return frame, predictions

def roboflow_inference(cap):
    """Run inference in real-time using the webcam."""
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame, predictions = perform_inference_and_update(cap)
        frame = update_predictions(frame, predictions)

        cv2.imshow('Roboflow Inference', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def handle_client(conn, addr):
    print(f"Accepted connection from {addr}")

    # Open the webcam and start inference
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise Exception("Could not open video device")

    # Start inference in a separate thread
    inference_thread = threading.Thread(target=roboflow_inference, args=(cap,))
    inference_thread.start()

    while True:
        try:
            # Receive data from the client
            data = conn.recv(1024)
            if not data:
                break

            # Process and send the response back to the client
            msg = bytes(data, 'utf-8')
            conn.send(msg)
        except Exception as e:
            print(f"Error handling connection from {addr}: {e}")
            break

    # Close the connection
    conn.close()
    print(f"Connection from {addr} closed")
    inference_thread.join()

def start_server():
    host = 'localhost'
    port = 5350

    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(5)

    print(f"Server listening on {host}:{port}")

    while True:
        try:
            conn, addr = server_socket.accept()
            client_thread = threading.Thread(target=handle_client, args=(conn, addr))
            client_thread.start()
        except Exception as e:
            print(f"Error accepting connection: {e}")

if __name__ == "__main__":
    start_server()


Server listening on localhost:5350
Accepted connection from ('127.0.0.1', 65503)
Error handling connection from ('127.0.0.1', 65503): [WinError 10054] An existing connection was forcibly closed by the remote host
Connection from ('127.0.0.1', 65503) closed
Accepted connection from ('127.0.0.1', 54257)
Error handling connection from ('127.0.0.1', 54257): [WinError 10054] An existing connection was forcibly closed by the remote host
Connection from ('127.0.0.1', 54257) closed
